In [1]:
import os
import sys
from pathlib import Path

os.chdir(Path(os.getcwd()).parents[0])
sys.path.append(os.getcwd())

import glob
import numpy as np
import torch
import pandas as pd

/home/l727n/anaconda3/envs/xai-eval/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
modality = "Voxel" # Image, Voxel, Point_Cloud
print(sorted(glob.glob("data/evaluation/" + modality + "/*"), key=os.path.getmtime))

file = sorted(glob.glob("data/evaluation/" + modality + "/*"), key=os.path.getmtime)[-1] # selects always the newsest
array = np.load(file, allow_pickle=True)
data = array['arr_0'] # models, xai, eval, obs

['data/evaluation/Voxel/eval_pn_AdrenalMNIST3D_dataset.npz', 'data/evaluation/Voxel/eval_pn_OrganMNIST3D_dataset.npz', 'data/evaluation/Voxel/eval_pn_VesselMNIST3D_dataset.npz']


In [20]:
attr_conv = np.stack([data[0], data[1]])
attr_tran = data[2][0:14]
attn = data[2][14:17]

In [21]:
attr_tran[:,8] # model, expl, eval, n

array([[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1], dtype=object)                                          ,
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1], dtype=object)                                          ,
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1], dtype=object)                                          ,
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
               0, 0, 0], dtype=object)                                          ,
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1], dtype=object)                                          ,
        array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1], dtype=object)                                          ,
        array([1, 1, 1

In [6]:
for i in range(attr_conv.shape[2]):
    print(np.sum(pd.isna(attr_conv[:,:,i,:])))

0
0
0
0
0
0
0
0
0
0
0
0
0
326
328
292
0
0
0
0
